<a href="https://colab.research.google.com/github/trhieu2/hands-on-ml-practices/blob/main/cafa6_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# TẠO MỘT ĐƯỜNG DẪN AN TOÀN TRÊN DRIVE CỦA BẠN
# (Bạn có thể tạo thư mục CAFA6_Project trong MyDrive trước)
SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs(SAVE_PATH, exist_ok=True)

print(f"Các file sẽ được lưu tại: {SAVE_PATH}")

Các file sẽ được lưu tại: /content/drive/MyDrive/CAFA6_Project/


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install obonet

In [ ]:
import obonet
import networkx as nx

In [ ]:
graph = obonet.read_obo('/content/drive/MyDrive/cafa6-data/Train/go-basic.obo')

nodes_data = []
for node_id, data in graph.nodes(data=True):
    node_info = {'id': node_id}
    node_info.update(data)
    nodes_data.append(node_info)

nodes_df = pd.DataFrame(nodes_data)

In [ ]:
nodes_df.head()

,id,name,namespace,def,synonym,is_a,xref,alt_id,subset,relationship,comment
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","[""mitochondrial inheritance"" EXACT []]","[GO:0048308, GO:0048311]",NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,[GO:0007005],NaN,NaN,NaN,NaN,NaN
2,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...","[""high affinity zinc uptake transmembrane tran...",[GO:0005385],NaN,NaN,NaN,NaN,NaN
3,GO:0000007,low-affinity zinc ion transmembrane transporte...,molecular_function,"""Enables the transfer of a solute or solutes f...",NaN,[GO:0005385],NaN,NaN,NaN,NaN,NaN
4,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,"""Catalysis of the transfer of a mannose residu...","[""1,6-alpha-mannosyltransferase activity"" EXAC...",[GO:0000030],"[Reactome:R-HSA-449718 ""Addition of a third ma...",NaN,NaN,NaN,NaN


In [ ]:
!pip install biopython

In [ ]:
from Bio import SeqIO

taxonomy_df = pd.read_csv('/content/drive/MyDrive/cafa6-data/Train/train_taxonomy.tsv', sep='\t', names=['EntryID', 'taxon ID'])
print(taxonomy_df.shape)

(82404, 2)


In [ ]:
taxonomy_df.head()

,EntryID,taxon ID
0,A0A0C5B5G6,9606
1,A0JNW5,9606
2,A0JP26,9606
3,A0PK11,9606
4,A1A4S6,9606


In [ ]:
sequences = list(SeqIO.parse('/content/drive/MyDrive/cafa6-data/Train/train_sequences.fasta', 'fasta'))
sequences[:1]

[SeqRecord(seq=Seq('MRWQEMGYIFYPRKLR'), id='sp|A0A0C5B5G6|MOTSC_HUMAN', name='sp|A0A0C5B5G6|MOTSC_HUMAN', description='sp|A0A0C5B5G6|MOTSC_HUMAN Mitochondrial-derived peptide MOTS-c OS=Homo sapiens OX=9606 GN=MT-RNR1 PE=1 SV=1', dbxrefs=[])]

In [ ]:
sequences_data = []
for seq in sequences:
    sequences_data.append({
        "EntryID": seq.id.split('|')[1],
        "Gene Name": seq.id.split('|')[2],
        "Amino-acid sequence": str(seq.seq),
        "Sequence length": len(seq.seq)
    })

sequences_df = pd.DataFrame(sequences_data)

sequences_df.head()

,EntryID,Gene Name,Amino-acid sequence,Sequence length
0,A0A0C5B5G6,MOTSC_HUMAN,MRWQEMGYIFYPRKLR,16
1,A0JNW5,BLT3B_HUMAN,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,1464
2,A0JP26,POTB3_HUMAN,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,581
3,A0PK11,CLRN2_HUMAN,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,232
4,A1A4S6,RHG10_HUMAN,MGLQPLEFSDCYLDSPWFRERIRAHEAELERTNKFIKELIKDGKNL...,786


In [ ]:
terms_df = pd.read_csv('/content/drive/MyDrive/cafa6-data/Train/train_terms.tsv', sep='\t')
print(terms_df.shape)

(537027, 3)


In [ ]:
terms_df.head()

,EntryID,term,aspect
0,Q5W0B1,GO:0000785,C
1,Q5W0B1,GO:0004842,F
2,Q5W0B1,GO:0051865,P
3,Q5W0B1,GO:0006275,P
4,Q5W0B1,GO:0006513,P


## Setup

In [ ]:
!pip install transformers accelerate

In [ ]:
import torch
from transformers import EsmTokenizer, EsmModel

model_name = "facebook/esm2_t30_150M_UR50D"
tokenizer = EsmTokenizer.from_pretrained(model_name)

model = EsmModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
print(f"Model loaded on: {device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Create embeddings

In [ ]:
from tqdm.auto import tqdm

def get_embeddings(sequence_batch):
    """
    Hàm này nhận vào một danh sách các chuỗi amino acid
    và trả về một numpy array chứa các embeddings.
    """
    # 1. Tokenize: Chuyển chuỗi thành các token (số) mà mô hình hiểu
    # padding=True: Thêm token "đệm" để các chuỗi trong batch dài bằng nhau
    # truncation=True: Cắt bớt các chuỗi quá dài (ESM có giới hạn)
    # max_length=1024: Một giới hạn an toàn cho độ dài chuỗi
    # return_tensors="pt": Trả về dưới dạng PyTorch Tensors
    inputs = tokenizer(
        sequence_batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    )

    # 2. Chuyển dữ liệu input sang GPU
    # Đây là bước quan trọng để tăng tốc!
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 3. Đưa qua mô hình
    # torch.no_grad() tắt việc tính toán đạo hàm (gradient),
    # giúp tiết kiệm bộ nhớ và tăng tốc vì chúng ta chỉ "dự đoán" (inference)
    with torch.no_grad():
        outputs = model(**inputs)

    # 4. Lấy embeddings
    # outputs.last_hidden_state có shape: (batch_size, sequence_length, embedding_dim)
    # Chúng ta lấy trung bình (mean) theo chiều sequence_length (dim=1)
    # để tạo ra 1 vector duy nhất cho mỗi protein
    # Shape cuối: (batch_size, embedding_dim)
    embeddings = outputs.last_hidden_state.mean(dim=1)

    # 5. Chuyển embeddings về CPU và sang dạng numpy để xử lý tiếp
    return embeddings.cpu().numpy()

### Test on small batch

In [ ]:
# Lấy 5 chuỗi đầu tiên từ DataFrame bạn đã tải
test_sequences = sequences_df['Amino-acid sequence'].head(5).tolist()

print("Bắt đầu tạo embedding cho 5 chuỗi thử nghiệm...")
test_embeddings = get_embeddings(test_sequences)

print("Hoàn thành!")
print(f"Shape của embeddings (Số lượng, Chiều vector): {test_embeddings.shape}")

### Run on riel set

In [ ]:
# Kích thước batch, bạn có thể thử tăng lên 128 nếu GPU cho phép
BATCH_SIZE = 64

# Danh sách để lưu trữ các batch embeddings
all_embeddings = []

# Lấy toàn bộ danh sách chuỗi
sequences = sequences_df['Amino-acid sequence'].tolist()

print(f"Bắt đầu tạo embeddings cho {len(sequences)} proteins...")
print(f"Tổng số batch: { (len(sequences) // BATCH_SIZE) + 1 }")

# Vòng lặp chính, đi qua danh sách theo từng BATCH_SIZE
# tqdm sẽ cho chúng ta một thanh tiến trình đẹp mắt
for i in tqdm(range(0, len(sequences), BATCH_SIZE)):
    # Lấy ra 1 batch
    batch_seqs = sequences[i:i + BATCH_SIZE]

    # Tạo embeddings cho batch đó
    batch_embeds = get_embeddings(batch_seqs)

    # Thêm vào danh sách
    all_embeddings.append(batch_embeds)

print("Đã tạo xong embeddings cho tất cả protein!")

# Ghép tất cả các batch lại thành một ma trận numpy lớn
full_embeddings = np.concatenate(all_embeddings, axis=0)

print(f"Shape của ma trận embeddings cuối cùng: {full_embeddings.shape}")

Bắt đầu tạo embeddings cho 82404 proteins...
Tổng số batch: 1288


  0%|          | 0/1288 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Save embeddings



In [ ]:
np.save(SAVE_PATH + 'train_embeddings_esm150M.npy', full_embeddings)

print("Đã lưu embeddings ra file 'train_embeddings_esm150M.npy'")

## Prepare labels

### Get GO terms list

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import joblib # Dùng để lưu binarizer

In [ ]:
all_terms = sorted(list(terms_df['term'].unique()))
num_classes = len(all_terms)

print(f"Tổng số lớp (GO term) duy nhất: {num_classes}")

### Group labels

In [ ]:
labels_grouped_df = terms_df.groupby('EntryID')['term'].apply(list).reset_index()
ordered_entry_ids_df = sequences_df[['EntryID']]
aligned_labels_df = ordered_entry_ids_df.merge(labels_grouped_df, on='EntryID', how='left')
aligned_labels_df['term'] = aligned_labels_df['term'].apply(lambda x: x if isinstance(x, list) else [])

print("DataFrame labels đã căn chỉnh (5 hàng đầu):")
print(aligned_labels_df.head())
print(f"Shape của DataFrame labels đã căn chỉnh: {aligned_labels_df.shape}")

### Binarize and store labels

In [ ]:
mlb = MultiLabelBinarizer(classes=all_terms)
labels_list = aligned_labels_df['term'].tolist()
full_labels_matrix = mlb.fit_transform(labels_list)

print(f"Shape Embeddings (X): {full_embeddings.shape}")
print(f"Shape Labels (y):     {full_labels_matrix.shape}")

In [ ]:
np.save(SAVE_PATH + 'train_labels.npy', full_labels_matrix)
print("Đã lưu ma trận labels (y) ra file 'train_labels.npy'")

joblib.dump(mlb, SAVE_PATH + 'multilabel_binarizer.joblib')
print("Đã lưu MultiLabelBinarizer (mlb) ra file 'multilabel_binarizer.joblib'")

## Model training

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

X = np.load('/content/drive/MyDrive/CAFA6_Project/train_embeddings_esm150M.npy')
y = np.load('/content/drive/MyDrive/CAFA6_Project/train_labels.npy')

print(f"Shape của X: {X.shape}")
print(f"Shape của y: {y.shape}")

Shape của X: (82404, 640)
Shape của y: (82404, 26125)


### Train-Test split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.1,
    random_state=42
)

### Construct a neural network

Input (640) -> Dense(1024) -> Dense(1024) -> Output (số_lớp)

In [ ]:
embedding_dim = X_train.shape[1]
num_classes = y_train.shape[1]

print(f"Input dimension (Embedding): {embedding_dim}")
print(f"Output dimension (Classes): {num_classes}")

# 1. Định nghĩa Input Layer
inputs = Input(shape=(embedding_dim,))

# 2. Xây dựng các lớp ẩn (Hidden Layers)
# Lớp 1
x = Dense(1024, activation='relu')(inputs)
x = BatchNormalization()(x) # Giúp ổn định quá trình học
x = Dropout(0.3)(x)         # Chống overfitting (học vẹt)

# Lớp 2
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# 3. Lớp Output (Đầu ra)
# Dùng activation 'sigmoid' vì đây là bài toán multi-label
# Mỗi neuron output sẽ cho ra xác suất (0 đến 1) cho 1 GO term
outputs = Dense(num_classes, activation='sigmoid')(x)

# 4. Tạo mô hình
model = Model(inputs, outputs)

### Compile

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=BinaryCrossentropy()
)

model.summary()

### Training


In [ ]:
checkpoint = ModelCheckpoint(
    SAVE_PATH + 'best_model.keras',        # Tên file để lưu mô hình
    monitor='val_loss',     # Theo dõi val_loss
    save_best_only=True,    # Chỉ lưu mô hình tốt nhất
    mode='min',             # Chế độ 'min' vì chúng ta muốn val_loss thấp nhất
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,             # Dừng nếu val_loss không cải thiện sau 3 epochs
    mode='min',
    verbose=1
)

In [ ]:
BATCH_SIZE = 256
EPOCHS = 20 # Bắt đầu với 20, EarlyStopping có thể sẽ dừng sớm hơn

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint, early_stopping]
)

## Submission file

In [ ]:
# Hàm đọc file FASTA đơn giản
def read_fasta(file_path):
    sequences = []
    entry_ids = []
    current_seq = ""
    current_id = ""

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                # Nếu có chuỗi cũ, lưu lại
                if current_id:
                    sequences.append(current_seq)
                    entry_ids.append(current_id)

                # Bắt đầu chuỗi mới
                current_id = line[1:].split()[0] # Lấy EntryID
                current_seq = ""
            else:
                current_seq += line

        # Đừng quên lưu chuỗi cuối cùng
        if current_id:
            sequences.append(current_seq)
            entry_ids.append(current_id)

    return pd.DataFrame({'EntryID': entry_ids, 'Amino-acid sequence': sequences})

print("Đang đọc file test_sequences.fasta...")
# Thay thế bằng đường dẫn đúng nếu cần
test_sequences_df = read_fasta('/content/drive/MyDrive/cafa6-data/Test/testsuperset.fasta')

print(f"Đã tải {len(test_sequences_df)} chuỗi protein từ tập test.")
print(test_sequences_df.head())

Đang đọc file test_sequences.fasta...
Đã tải 224309 chuỗi protein từ tập test.
      EntryID                                Amino-acid sequence
0  A0A0C5B5G6                                   MRWQEMGYIFYPRKLR
1  A0A1B0GTW7  MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...
2      A0JNW5  MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...
3      A0JP26  MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...
4      A0PK11  MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...


In [ ]:
import os
import glob
import numpy as np
from tqdm.auto import tqdm

# --- ĐẢM BẢO CÁC BIẾN NÀY ĐÃ TỒN TẠI ---
# 1. SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'
# 2. test_sequences_df (đã đọc từ file fasta)
# 3. Hàm get_embeddings()
# 4. Biến 'model' là MÔ HÌNH ESM-2 (đã tải lại từ Bước 1)
# ----------------------------------------------------

BATCH_SIZE = 64 # Bạn có thể dùng 128 nếu GPU mạnh

# Tạo một thư mục trên DRIVE của bạn để lưu các batch
output_dir = os.path.join(SAVE_PATH, 'test_embeddings_batches')
os.makedirs(output_dir, exist_ok=True)

print(f"Thư mục lưu batch test: {output_dir}")

# Lấy danh sách chuỗi
test_sequences = test_sequences_df['Amino-acid sequence'].tolist()
print(f"Bắt đầu tạo embeddings cho {len(test_sequences)} protein test...")

# --- VÒNG LẶP AN TOÀN ---
for i in tqdm(range(0, len(test_sequences), BATCH_SIZE), desc="Tạo test embedding"):

    batch_index = i // BATCH_SIZE
    # File path trỏ thẳng vào Google Drive
    file_path = os.path.join(output_dir, f'batch_{batch_index:04d}.npy')

    # *** PHẦN SỬA QUAN TRỌNG: BỎ QUA NẾU ĐÃ LƯU ***
    # Nếu file batch này đã tồn tại, bỏ qua và sang vòng lặp tiếp theo
    if os.path.exists(file_path):
        continue

    # Lấy batch và tạo embedding
    batch_seqs = test_sequences[i:i + BATCH_SIZE]
    batch_embeds = get_embeddings(batch_seqs)

    # *** PHẦN SỬA QUAN TRỌNG: LƯU BATCH NGAY LẬP TỨC ***
    # Lưu ngay batch này vào Google Drive
    np.save(file_path, batch_embeds)

print("Đã hoàn thành tạo tất cả các batch embeddings!")

# --- GHÉP TẤT CẢ CÁC FILE BATCH LẠI ---
print("Đang ghép các file batch từ Google Drive...")

# Tìm tất cả các file .npy trong thư mục batch
# Sắp xếp lại để đảm bảo đúng thứ tự
all_files = sorted(glob.glob(os.path.join(output_dir, '*.npy')))

# Tải từng file batch và cho vào list
all_batches = []
for f in tqdm(all_files, desc="Đang tải các batch đã lưu"):
    all_batches.append(np.load(f))

# Ghép lại thành 1 ma trận numpy lớn
test_embeddings = np.concatenate(all_batches, axis=0)

print("--- KIỂM TRA SHAPE ---")
print(f"Số lượng protein trong test_sequences_df: {len(test_sequences_df)}")
print(f"Shape của ma trận test embeddings cuối cùng: {test_embeddings.shape}")
# (Hai con số đầu tiên phải khớp nhau)

# *** PHẦN SỬA QUAN TRỌNG: LƯU FILE CUỐI VÀO DRIVE ***
final_test_embedding_file = os.path.join(SAVE_PATH, 'test_embeddings_esm150M.npy')
np.save(final_test_embedding_file, test_embeddings)
print(f"Đã lưu file test embeddings tổng vào: {final_test_embedding_file}")

In [ ]:
model_file = os.path.join(SAVE_PATH, 'best_model.keras')
test_embedding_file = os.path.join(SAVE_PATH, 'test_embeddings_esm150M.npy')

print(f"Đang tải mô hình đã huấn luyện từ: {model_file}")
# Tải mô hình đã huấn luyện
model = tf.keras.models.load_model(model_file)

print(f"Đang tải test embeddings từ: {test_embedding_file}")
# Tải ma trận test embeddings
test_embeddings = np.load(test_embedding_file)

print(f"Shape của test embeddings: {test_embeddings.shape}")

Đang tải mô hình đã huấn luyện từ: /content/drive/MyDrive/CAFA6_Project/best_model.keras
Đang tải test embeddings từ: /content/drive/MyDrive/CAFA6_Project/test_embeddings_esm150M.npy
Shape của test embeddings: (224309, 640)


In [ ]:
print("Bắt đầu dự đoán (predict) trên tập test...")
# Đây là ma trận xác suất (shape: [num_test_proteins, num_go_terms])
test_predictions = model.predict(
    test_embeddings,
    batch_size=BATCH_SIZE * 4 # Có thể tăng batch_size khi predict để nhanh hơn
)

print(f"Shape của ma trận dự đoán: {test_predictions.shape}")

Bắt đầu dự đoán (predict) trên tập test...
877/877 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step
Shape của ma trận dự đoán: (224309, 26125)


In [ ]:
import joblib
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

print("--- Bước 5.3 (Phiên bản TỐI ƯU RAM) ---")

# --- ĐẢM BẢO CÁC BIẾN NÀY ĐÃ TỒN TẠI ---
# 1. SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'
# 2. test_sequences_df (DataFrame chứa EntryID)
# 3. test_predictions (Ma trận numpy từ Bước 5.2)
# ----------------------------------------------------

# 1. Tải các tài nguyên cần thiết
print("Đang tải binarizer và lấy danh sách EntryID...")
binarizer_file = os.path.join(SAVE_PATH, 'multilabel_binarizer.joblib')
mlb = joblib.load(binarizer_file)
go_term_columns = mlb.classes_
test_entry_ids = test_sequences_df['EntryID'].tolist()

# 2. Định nghĩa file output và kích thước chunk
submission_file_path = 'submission.tsv'
CHUNK_SIZE = 5000 # Xử lý 5000 protein mỗi lần (bạn có thể tăng nếu RAM vẫn còn)
num_chunks = (len(test_predictions) // CHUNK_SIZE) + 1

print(f"Sẽ xử lý {len(test_predictions)} protein theo {num_chunks} chunks...")

# 3. Vòng lặp xử lý từng chunk
for i in tqdm(range(0, len(test_predictions), CHUNK_SIZE), desc="Xử lý và lưu chunks"):

    # Lấy ra một chunk (lô) dự đoán và ID
    start_idx = i
    end_idx = min(i + CHUNK_SIZE, len(test_predictions))

    preds_chunk = test_predictions[start_idx:end_idx]
    ids_chunk = test_entry_ids[start_idx:end_idx]

    # Tạo DataFrame "wide" CHỈ CHO CHUNK NÀY (nhỏ, vừa RAM)
    chunk_df_wide = pd.DataFrame(preds_chunk, columns=go_term_columns)
    chunk_df_wide['EntryID'] = ids_chunk

    # Melt CHỈ CHUNK NÀY
    chunk_df_long = chunk_df_wide.melt(
        id_vars=['EntryID'],
        var_name='term',
        value_name='probability'
    )

    # Lọc bỏ xác suất thấp NGAY LẬP TỨC
    chunk_df_long = chunk_df_long[chunk_df_long['probability'] > 0.01]

    # Ghi vào file
    if i == 0:
        # Lần chạy đầu tiên, ghi đè file (write mode 'w') và KHÔNG có header
        chunk_df_long.to_csv(
            submission_file_path,
            sep='\t',
            index=False,
            header=False,
            mode='w'
        )
    else:
        # Các lần sau, ghi nối tiếp (append mode 'a') và KHÔNG có header
        chunk_df_long.to_csv(
            submission_file_path,
            sep='\t',
            index=False,
            header=False,
            mode='a'
        )

print("--- HOÀN TẤT TOÀN BỘ QUY TRÌNH! ---")
print(f"File '{submission_file_path}' đã được tạo thành công (tối ưu RAM).")

--- Bước 5.3 (Phiên bản TỐI ƯU RAM) ---
Đang tải binarizer và lấy danh sách EntryID...
Sẽ xử lý 224309 protein theo 45 chunks...


Xử lý và lưu chunks:   0%|          | 0/45 [00:00<?, ?it/s]

--- HOÀN TẤT TOÀN BỘ QUY TRÌNH! ---
File 'submission.tsv' đã được tạo thành công (tối ưu RAM).
